In [13]:
from selenium.webdriver.common.by import By
from datetime import datetime
import selenium as sel
from tqdm import tqdm
import pandas as pd
import base as bs
import os as os

In [14]:
#Empresa con la cual vamos a extraer los articulos
empresa = input("Digite la empresa a extraer: ").lower().strip()
if " " in empresa:
    empresa_ = empresa.replace(" ", "%20")
else:
    empresa_ = empresa
revista = "elColombiano"

In [15]:
# crear driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.elcolombiano.com/busqueda/-/search/{empresa_}/false/false/19810311/20230311/date/true/true/0/0/meta/0/0/0/1')
driver.maximize_window()

In [ ]:
# Articulos a extraer
num_paginas = input("Ingrese el numero de páginas que desea recorrer").lower()
url_princ = driver.current_url[:-1]
titulares = []
for i in tqdm(range(1, int(num_paginas) + 1)):
    aux = str(i)
    url_a_buscar = url_princ+aux
    driver.get(url_a_buscar)
    articulos = driver.find_elements(By.XPATH, './/li[@class="element   full-access norestricted"]')
    
    for art in tqdm(articulos):
        url = art.find_element(By.XPATH, './/div[contains(@class, "right")]//a').get_attribute('href')
        if not(bs.existedb(url, "elColombiano")):
            titulo = art.find_element(By.XPATH, './/h3[contains(@class, "titulo-noticia")]//span').text
            fechaP = art.find_element(By.XPATH, './/div[contains(@class, "fecha")]//span').text
            fechaP = datetime.strptime(fechaP, '%d / %m / %Y')
            tema = art.find_element(By.XPATH, './/div[contains(@class, "information")]//a').text
            autor = art.find_element(By.XPATH, './/span[contains(@class, "autor")]').text
            lista_tags = art.find_element(By.XPATH, './/div[contains(@class, "tags-noticia")]//ul')
            tags = list(map(lambda x: x.text, lista_tags.find_elements(By.XPATH, './/li')))
            imagen = bs.obtener_imagen_col(art)
            titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                            'Titulo': titulo,
                            'Fecha Publicacion': fechaP,
                            'URL': url,
                            'Tema': tema,
                            'Autor': autor,
                            'Tags': tags,
                            'Imagen': imagen,
                            'Empresa': empresa,
                            'Fuente': 'El Colombiano'
                            })

In [ ]:
for tit in titulares:
    driver.get(tit["URL"])
    driver.implicitly_wait(10)
    tit["Contenido"] = bs.obtener_contenido_col(driver)
    tit["Resumen"] = bs.obtener_resumen_col(driver)

In [ ]:
df = pd.DataFrame(titulares)
bs.writeData("database", df)

,Fecha Extraccion,Titulo,Fecha Publicacion,URL,Tema,Autor,Tags,Imagen,Empresa,Fuente,Contenido,Resumen
0,2023-03-28 09:38:00,Colombia 2050,2023-03-10,https://www.elcolombiano.com/opinion/columnist...,COLUMNISTAS,JUAN CARLOS MANRIQUE,[],https://www.elcolombiano.com/binrepository/147...,ecopetrol,El Colombiano,,None
1,2023-03-28 09:38:00,La gran paradoja\nde las empresas,2023-03-09,https://www.elcolombiano.com/opinion/editorial...,EDITORIALES,,[],None,ecopetrol,El Colombiano,,None
2,2023-03-28 09:38:01,Ecopetrol firma alianza para impulsar la energ...,2023-03-08,https://www.elcolombiano.com/negocios/ecopetro...,ECONOMÍA,ALEJANDRA ZAPATA QUINCHÍA,"[ECONOMÍA, EMPRESAS, ECOPETROL, SECTORES ECONÓ...",https://www.elcolombiano.com/binrepository/156...,ecopetrol,El Colombiano,"Ecopetrol, Baker Hughes y la Central Hidroeléc...",El desarrollo de la iniciativa permitiría gene...
3,2023-03-28 09:38:01,"Por inestabilidad, importante petrolera suspen...",2023-03-08,https://www.elcolombiano.com/negocios/emerald-...,ECONOMÍA,DIARIO LA REPÚBLICA,"[EMPRESAS, SECTORES ECONÓMICOS, HIDROCARBUROS,...",https://www.elcolombiano.com/binrepository/162...,ecopetrol,El Colombiano,"La semana pasada Francisco José Lloreda, presi...","Se trata de Emerald Energy, que tenía prevista..."
4,2023-03-28 09:38:01,Encontraron manatí muerto en aguas de influenc...,2023-03-07,https://www.elcolombiano.com/colombia/manati-m...,COLOMBIA,JUAN PABLO PATIÑO,"[MEDIO AMBIENTE, ANIMALES, COLOMBIA, SANTANDER]",https://www.elcolombiano.com/binrepository/88x...,ecopetrol,El Colombiano,Los pescadores del corregimiento de El Llanito...,Una unidad forense establecerá las causas de l...


In [ ]:
driver.close()